In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import time

In [80]:
url = 'https://store.steampowered.com/search/results/?query&start=0&count=50&dynamic_data=&sort_by=_ASC&tags=4182&snr=1_7_7_2300_7&specials=1&filter=topsellers&infinite=1'

def totalresults(url):
    r = requests.get(url)
    data = dict(r.json())
    totalresults = data['total_count']
    return int(totalresults)

def get_data(url):
    r = requests.get(url)
    data = dict(r.json())
    return data['results_html']

#print (get_data(url))

In [81]:
def parse(data):
    gameslist = []
    soup = BeautifulSoup(data, 'html.parser')
    games = soup.find_all('a')
    for game in games:
        title = game.find('span', {'class': 'title'}).text
        try:
            date_released = game.find('div', {'class' : 'search_released'}).text
        except:
            date_released = ' '
        original_price = game.find('div', {'class': 'search_price'}).text.strip().split('RM')[1]
        try:
            discount_price = game.find('div', {'class': 'search_price'}).text.strip().split('RM')[2]
        except:
            discount_price = original_price
        try:
            review = game.find('span', {'data-tooltip-html': True})["data-tooltip-html"]
        except:
            review = ' '
        #print(title, date_released, original_price, discount_price, review)

        mygame ={
            'title': title,
            'date_released': date_released,
            'original_price': original_price,
            'discount_price': discount_price,
            'review': review
        }
        gameslist.append(mygame)
    return gameslist

In [82]:
def output(results):
    gamesdf = pd.concat([pd.DataFrame(g) for g in results])
    gamesdf.to_csv('SteamTopSellerSpecialOfferPrices_Singleplayer.csv', index=False)
    print('Fin. Saved to CSV')
    print(gamesdf.head())
    return

In [84]:
results = []
for x in range(0, totalresults(url), 50):
    data = get_data(f'https://store.steampowered.com/search/results/?query&start={x}&count=50&dynamic_data=&sort_by=_ASC&tags=4182&snr=1_7_7_2300_7&specials=1&filter=topsellers&infinite=1')
    results.append(parse(data))
    #print('Results Scraped: ', x)
    time.sleep(1.5)

In [85]:
output(results)

Fin. Saved to CSV
                                 title date_released original_price  \
0                   Battlefield Bundle                       567.00   
1  Fallout 4: Game of the Year Edition  10 Nov, 2015         167.48   
2                        Battlefield V  22 Oct, 2020         249.00   
3        Disco Elysium - The Final Cut  15 Oct, 2019          65.00   
4              Gladiator Guild Manager  13 Oct, 2021          25.00   

  discount_price                                             review  
0          47.29  Mostly Positive<br>78% of the 129,057 user rev...  
1          41.87  Mostly Positive<br>78% of the 170,391 user rev...  
2          24.90  Mixed<br>69% of the 59,724 user reviews for th...  
3          35.75  Very Positive<br>94% of the 38,483 user review...  
4          22.50  Very Positive<br>89% of the 77 user reviews fo...  
